In [3]:
import json
import numpy as np
from openai import OpenAI
from typing import List
import faiss
import pickle
from dotenv import load_dotenv

In [6]:
# 환경 변수 설정 (.env 또는 직접 설정)
load_dotenv()

client = OpenAI()

filename = ["skinner"]
# 2. 텍스트 파싱
raw_text = []
for n in filename:
    with open(f"{n}.json", "r", encoding="utf-8") as f:
        raw_text = json.load(f)

In [8]:
docs = [text["content"] for text in raw_text]
metadatas = [{k: v for k, v in text.items() if k != "content"} for text in raw_text]

print(docs[2])

조절은 아동이 새로운 정보를 이해하기 위해 기존 스키마를 수정하거나 새로운 스키마를 창출하는 인지적 과정이다. 이는 기존 스키마로는 더 이상 설명이 불가능한 경험이 주어졌을 때, 사고 구조를 재구성하는 방식으로 작동한다. 조절은 기존의 틀을 유지하려는 동화와 달리, 자신의 사고방식을 유연하게 바꾸는 능동적인 학습 과정이다. 예를 들어, 아동이 고양이를 처음 보고 강아지라고 했다가 '이건 고양이야'라는 피드백을 통해 '고양이'라는 새로운 스키마를 형성하게 되는 것이 조절이다. 조절은 기존의 이해 구조에 충격을 주는 정보를 마주했을 때 일어나며, 이는 인지적 균형 상태를 다시 찾기 위한 자연스러운 반응이다. 피아제는 조절이 없이는 진정한 학습과 발달이 일어날 수 없다고 보았다. 조절은 아동의 인지 구조가 더 정교해지고 현실 세계에 보다 정확하게 적응하도록 돕는다.


In [9]:
# OpenAI Embedding
def get_embeddings(texts: List[str]) -> List[List[float]]:
    response = client.embeddings.create(input=texts, model="text-embedding-3-small")
    return [e.embedding for e in response.data]


embeddings = get_embeddings(docs)

# FAISS 벡터스토어 저장
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))

# 텍스트 + 메타데이터 함께 저장
chunks = [{"doc": doc, **meta} for doc, meta in zip(docs, metadatas)]

with open("faiss_chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

faiss.write_index(index, "faiss_index.idx")
print("✅ 벡터스토어 저장 완료")

✅ 벡터스토어 저장 완료


In [10]:
# 5. 검색 함수
def search(query: str, k: int = 3):
    q_embedding = get_embeddings([query])[0]
    D, I = index.search(np.array([q_embedding]).astype("float32"), k)
    with open("faiss_chunks.pkl", "rb") as f:
        stored_chunks = pickle.load(f)
    return [stored_chunks[i] for i in I[0]]

In [ ]:
results = search("우리 아이가 자주 아픈데, 어떻게 해야 할까요?", k=3)
for r in results:
    print(r["author"], r["title"])
    print(r["doc"])

Jean Piaget 스키마 (Schema)
스키마는 아동이 외부 세계를 인식하고 해석하며 반응하는 방식을 조직화하는 인지적 틀이다. 피아제는 스키마를 '행동적 혹은 정신적 구조물'로 보았으며, 이는 특정 자극이나 상황에 대해 일관된 방식으로 사고하거나 행동하도록 만드는 내부 체계라고 정의했다. 초기에는 단순한 감각운동적 반응(예: 빠는 행동, 쥐는 행동) 수준의 스키마로 시작하지만, 아동이 경험을 축적하면서 점차 복잡한 인지적 스키마로 발전한다. 스키마는 단지 정보를 저장하는 그릇이 아니라, 새로운 자극을 이해하고 예측하게 만드는 능동적인 틀이며, 인지 발달의 출발점이자 핵심 구조이다. 예를 들어, 아동이 '공'이라는 스키마를 가지고 있다면, 새로 접하는 둥글고 굴러가는 물체도 그 스키마 안에서 해석하고 반응하게 된다. 스키마는 학습과 경험을 통해 계속해서 재조직화되며, 이 과정에서 동화와 조절이 핵심 역할을 한다.
Jean Piaget 동화 (Assimilation)
동화는 아동이 외부 세계로부터 들어오는 새로운 정보를 기존의 스키마에 맞추어 해석하고 통합하는 과정이다. 즉, 이미 가지고 있는 사고 틀을 바탕으로 새로운 자극을 흡수하려는 인지적 작용이다. 이는 외부 세계를 기존 이해 방식에 일치시키는 방식으로 작동하며, 익숙한 상황에서는 매우 효율적인 학습 전략이 된다. 예를 들어, 네발 달린 동물을 모두 '강아지'로 인식하는 아동은 고양이나 양도 강아지로 분류하는 오류를 범할 수 있다. 이는 새로운 정보를 기존 스키마에 맞춰 이해하려는 동화의 전형적 사례다. 동화는 아동이 세상을 더 빠르게 이해하도록 돕지만, 모든 정보가 동화될 수 있는 것은 아니며, 기존 스키마로 설명이 불가능한 정보에 직면할 경우 조절이 필요해진다. 피아제는 동화를 통해 기존 지식이 강화되고, 반복되는 경험 속에서 점차 수정되는 과정을 통해 인지적 성장이 일어난다고 보았다.
Jean Piaget 평형 (Equilibration)
평형은 아동이 새로운 정보를 받아들이는 과정에서 동화와 